In [1]:
from dataclasses import dataclass
import torch
from typing import Dict
import numpy as np
import os

# Test Dataloader


In [2]:
from dataloader import create_dataloaders
from torchvision.transforms import functional as TF
import random

# Constants
PATH = '../data/Splitted CIFAR10.npz'

# Inits
transforms = {
    'random_horizontal_flip': lambda img: TF.hflip(img) if random.random() > 0.5 else img,
    'random_vertical_flip': lambda img: TF.vflip(img) if random.random() > 0.5 else img,
    'color_jitter': lambda img: TF.adjust_brightness(img, brightness_factor=random.uniform(0.8, 1.2)),
    'normalize': lambda img: TF.normalize(img, mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])
}

@dataclass
class DataConfig:
    npz_path_: str = '../data/Splitted CIFAR10.npz'
    lower_ucc: int = 2
    upper_ucc: int = 4
    bag_size: int = 300
    bag_fraction: float = 0.3
    batch_size: int = 32
    transform: Dict = None

# TEST 

# data_config_test = DataConfig()
# dataloaders = create_dataloaders(**data_config_test.__dict__)
# 
# # Testing the dataloaders
# for images, labels in dataloaders['train']:
#     print(f'Images batch shape: {images.shape}')
#     print(f'Labels batch shape: {labels.shape}')
#     print(f'Labels: {labels}')
#     print(f'Images: {images}')
#     break

# Test Model

In [3]:
from model import UCCModel

@dataclass
class ModelConfig:
    num_bins: int = 10
    sigma : float = 0.1
    dropout_rate: float = 0.1
    num_classes: int = 10
    embedding_size: int = 110
    fc2_size: int = 512

# Init    
model_config_test = ModelConfig()
model = UCCModel(**model_config_test.__dict__)

# Test

# # Mock data
# batch_size, num_instances, channels, height, width = 2, 5, 3, 32, 32
# random_data = torch.randn((batch_size, num_instances, channels, height, width))
# 
# # Forward pass through the model
# logits, decoded_imgs = model(random_data)
# 
# # Outputs
# print("Random Data:", random_data.shape)
# print("Logits shape:", logits.shape)
# print("Decoded images shape:", decoded_imgs.shape)
# print(model)

# Test Training Functions

In [4]:
from train import Trainer
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn


@dataclass
class TrainConfig:
    model: nn.Module
    optimizer: torch.optim.Optimizer
    train_loader: DataLoader
    val_loader: DataLoader
    model_name: str
    total_steps: int = 10_000
    eval_interval: int = 100
    ucc_loss_weight: float = 0.5
    model_dir: str = "./models"
    device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Test Pipeline


In [5]:
############################################
# CONFIGS
############################################
transforms = {
    'random_horizontal_flip': lambda img: TF.hflip(img) if random.random() > 0.5 else img,
    'random_vertical_flip': lambda img: TF.vflip(img) if random.random() > 0.5 else img,
    'color_jitter': lambda img: TF.adjust_brightness(img, brightness_factor=random.uniform(0.8, 1.2)),
    'normalize': lambda img: TF.normalize(img, mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])
}

DATA_CONFIG_1 = DataConfig(
    npz_path_ = PATH,
    lower_ucc = 1,
    upper_ucc = 4,
    bag_size = 20,
    bag_fraction = 0.2,
    batch_size = 10,
    transform = transforms
)

MODEL_CONFIG_1 = ModelConfig(
    num_bins = 10,
    sigma = 0.1,
    dropout_rate = 0.5,
    num_classes = 5,
    embedding_size = 50,
    fc2_size = 512
)
LEARNING_RATE = 0.0001

############################################
# Instantiate Parts
############################################

# data
dataloaders = create_dataloaders(**DATA_CONFIG_1.__dict__)

# model
model = UCCModel(**MODEL_CONFIG_1.__dict__)

# Optimizer 
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

############################################
# Train
############################################

TRAIN_CONFIG_1 = TrainConfig(
    model=model,
    optimizer=optimizer,
    train_loader=dataloaders['train'],
    val_loader=dataloaders['val'],
    model_name="test_model",
    total_steps=100,
    eval_interval=5,
    ucc_loss_weight=0.5,
    model_dir="../models",
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)
print(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
trainer = Trainer(**TRAIN_CONFIG_1.__dict__)

# TEST
trainer.train()

cuda
##########################
# Starting Training...
#########################


Training Progress:  95%|▉| 95/100 [00:16<00:00,  5.89it/s, Train Loss: 1.1187, V

Training completed. Best validation accuracy: 0.5571


# Experiments


In [ ]:
"""
EXPERIMENT CARD 

Config A
- num_bins: 10
- sigma : 0.1
- dropout_rate: 0.3
- num_classes: 5
- embedding_size: 128
- fc2_size: 256

The hypothesis here is that a moderate number of bins and a standard sigma value provide a good starting point for KDE. The dropout rate is set at a typical middle ground to prevent overfitting. An embedding size of 128 and fc2 size of 256 are assumed to provide enough capacity to capture complex relationships in the data without overly complexifying the model.
"""

MODEL_NAME = "UCC1to4_ExperimentA"

############################################
# CONFIGS
############################################
transforms = {
    'random_horizontal_flip': lambda img: TF.hflip(img) if random.random() > 0.5 else img,
    'random_vertical_flip': lambda img: TF.vflip(img) if random.random() > 0.5 else img,
    'color_jitter': lambda img: TF.adjust_brightness(img, brightness_factor=random.uniform(0.8, 1.2)),
    'normalize': lambda img: TF.normalize(img, mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])
}
DATA_CONFIG_1 = DataConfig(
    npz_path_ = PATH,
    lower_ucc = 1,
    upper_ucc = 4,
    bag_size = 100,
    bag_fraction = 0.9,
    batch_size = 10,
    transform = transforms
)
MODEL_CONFIG_1 = ModelConfig(
    num_bins = 10,
    sigma = 0.1,
    dropout_rate = 0.3,
    num_classes = 5,
    embedding_size = 128,
    fc2_size = 256
)
LEARNING_RATE = 0.0001

############################################
# Instantiate Parts
############################################

dataloaders = create_dataloaders(**DATA_CONFIG_1.__dict__)
model = UCCModel(**MODEL_CONFIG_1.__dict__)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

############################################
# Train
############################################
TRAIN_CONFIG_1 = TrainConfig(
    model=model,
    optimizer=optimizer,
    train_loader=dataloaders['train'],
    val_loader=dataloaders['val'],
    model_name=MODEL_NAME,
    total_steps=10_000,
    eval_interval=100,
    ucc_loss_weight=0.5,
    model_dir="../models",
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)

torch.cuda.empty_cache()
trainer = Trainer(**TRAIN_CONFIG_1.__dict__)
trainer.train()


##########################
# Starting Training...
#########################


Training Progress:  34%|▎| 3400/10000 [17:10<32:44,  3.36it/s, Train Loss: nan, 

In [ ]:
"""
EXPERIMENT CARD 

Config B
- num_bins: 20
- sigma : 0.05
- dropout_rate: 0.5
- num_classes: 5
- embedding_size: 256
- fc2_size: 128

Doubling the number of bins might capture more nuances in data distribution, and halving sigma could provide finer estimation. A higher dropout rate is chosen to combat potential overfitting due to the increased complexity from more bins. The larger embedding size is to capture more detailed features, while a smaller fc2 size is to test if a bottleneck improves generalization.
"""

MODEL_NAME = "UCC1to4_ExperimentB"

############################################
# CONFIGS
############################################
transforms = {
    'random_horizontal_flip': lambda img: TF.hflip(img) if random.random() > 0.5 else img,
    'random_vertical_flip': lambda img: TF.vflip(img) if random.random() > 0.5 else img,
    'color_jitter': lambda img: TF.adjust_brightness(img, brightness_factor=random.uniform(0.8, 1.2)),
    'normalize': lambda img: TF.normalize(img, mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])
}
DATA_CONFIG_1 = DataConfig(
    npz_path_ = PATH,
    lower_ucc = 1,
    upper_ucc = 4,
    bag_size = 100,
    bag_fraction = 0.5,
    batch_size = 64,
    transform = transforms
)
MODEL_CONFIG_1 = ModelConfig(
    num_bins = 20,
    sigma = 0.05,
    dropout_rate = 0.5,
    num_classes = 5,
    embedding_size = 256,
    fc2_size = 128
)
LEARNING_RATE = 0.0001

############################################
# Instantiate Parts
############################################

dataloaders = create_dataloaders(**DATA_CONFIG_1.__dict__)
model = UCCModel(**MODEL_CONFIG_1.__dict__)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

############################################
# Train
############################################
TRAIN_CONFIG_1 = TrainConfig(
    model=model,
    optimizer=optimizer,
    train_loader=dataloaders['train'],
    val_loader=dataloaders['val'],
    model_name=MODEL_NAME,
    total_steps=5_000,
    eval_interval=100,
    ucc_loss_weight=0.5,
    model_dir="../models",
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)

torch.cuda.empty_cache()
trainer = Trainer(**TRAIN_CONFIG_1.__dict__)
trainer.train()


In [ ]:
"""
EXPERIMENT CARD 

Config C
- num_bins: 5
- sigma : 0.2
- dropout_rate: 0.5
- num_classes: 5
- embedding_size: 64
- fc2_size: 512

With fewer bins and a larger sigma, the hypothesis is that the model might generalize better by avoiding capturing noise in the data distribution. A lower dropout rate is tested due to the simpler model. A smaller embedding is to see if a more compact representation suffices, while a larger fc2 size is to see if it allows for better classification boundaries.
"""

MODEL_NAME = "UCC1to4_ExperimentC"

############################################
# CONFIGS
############################################

MODEL_CONFIG_1 = ModelConfig(
    num_bins = 5,
    sigma = 0.2,
    dropout_rate = 0.5,
    num_classes = 5,
    embedding_size = 64,
    fc2_size = 512
)
LEARNING_RATE = 0.0001

############################################
# Instantiate Parts
############################################

dataloaders = create_dataloaders(**DATA_CONFIG_1.__dict__)
model = UCCModel(**MODEL_CONFIG_1.__dict__)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

############################################
# Train
############################################
TRAIN_CONFIG_1 = TrainConfig(
    model=model,
    optimizer=optimizer,
    train_loader=dataloaders['train'],
    val_loader=dataloaders['val'],
    model_name=MODEL_NAME,
    total_steps=5_000,
    eval_interval=100,
    ucc_loss_weight=0.5,
    model_dir="../models",
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)

torch.cuda.empty_cache()
trainer = Trainer(**TRAIN_CONFIG_1.__dict__)
trainer.train()
